
# Implementing Custom Wrappers

在本教程中，我们将描述如何实现您自己的自定义包装器。包装器是以模块化方式向环境添加功能的好方法。这将为您节省大量样板代码。
我们将展示如何创建包装器

- Inheriting from :class:`gymnasium.ObservationWrapper`
- Inheriting from :class:`gymnasium.ActionWrapper`
- Inheriting from :class:`gymnasium.RewardWrapper`
- Inheriting from :class:`gymnasium.Wrapper`

Before following this tutorial, make sure to check out the docs of the :mod:`gymnasium.wrappers` module.


## Inheriting from :class:`gymnasium.ObservationWrapper`

如果您想将某些函数应用于环境返回的观察结果，则观察包装器非常有用。如果您实现观察包装器，则只需通过实现来定义此转换 gymnasium.ObservationWrapper.observation() 方法。此外，如果转换改变了观察的形状（例如，通过将字典转换为 numpy 数组，如以下示例所示），您应该记住更新观察空间。

想象一下，您有一个 2D 导航任务，其中环境返回字典作为带有键"agent_position"和"target_position"的观察结果。一个常见的做法可能是丢弃某些程度的 自由度，只考虑目标相对于智能体的位置，即 observation["target_position"] - observation["agent_position"] 。为此，您可以实现如下观察包装器：

In [3]:
import numpy as np

import gymnasium as gym
from gymnasium.spaces import Box, Discrete
from gymnasium import ActionWrapper, ObservationWrapper, RewardWrapper, Wrapper


class RelativePosition(ObservationWrapper):
    def __init__(self, env):
        super().__init__(env)
        self.observation_space = Box(shape=(2,), low=-np.inf, high=np.inf)

    def observation(self, obs):
        return obs["target"] - obs["agent"]

## Inheriting from :class:`gymnasium.ActionWrapper`

可用于在将操作应用到环境之前对操作应用转换。 如果您实现操作包装器，则需要通过实现来定义该转换 gymnasium.ActionWrapper.action() 。此外，您应该通过更新包装器的操作空间来指定该转换的域。

假设您有一个动作空间类型为gymnasium.spaces.Box的环境，但您只想使用动作的有限子集。然后，您可能想要实现以下包装器：

In [5]:
class DiscreteActions(ActionWrapper):
    def __init__(self, env, disc_to_cont):
        super().__init__(env)
        self.disc_to_cont = disc_to_cont
        self.action_space = Discrete(len(disc_to_cont))

    def action(self, act):
        return self.disc_to_cont[act]


if __name__ == "__main__":
    env = gym.make("LunarLanderContinuous-v3")
    wrapped_env = DiscreteActions(
        env, [np.array([1, 0]), np.array([-1, 0]), np.array([0, 1]), np.array([0, -1])]
    )
    print(wrapped_env.action_space)  # Discrete(4)

Discrete(4)


## Inheriting from :class:`gymnasium.RewardWrapper`

奖励包装器用于转换环境返回的奖励。 对于前面的包装器，您需要通过实现来指定该转换 gymnasium.RewardWrapper.reward() 方法。

让我们看一个例子：有时（特别是当我们无法控制奖励，因为它是内在的），我们希望将奖励限制在一个范围内以获得一定的数值​​稳定性。为此，我们可以实现以下包装器：

In [6]:
from typing import SupportsFloat


class ClipReward(RewardWrapper):
    def __init__(self, env, min_reward, max_reward):
        super().__init__(env)
        self.min_reward = min_reward
        self.max_reward = max_reward

    def reward(self, r: SupportsFloat) -> SupportsFloat:
        return np.clip(r, self.min_reward, self.max_reward)

## Inheriting from :class:`gymnasium.Wrapper`

有时您可能需要实现一个包装器来进行一些更复杂的修改（例如，根据info中的数据修改奖励或更改渲染行为）。
这样的包装器可以通过继承gymnasium.Wrapper来实现。

您可以通过在__init__中分别定义self.action_space或self.observation_space来设置新的操作或观察空间

您可以通过在__init__中定义self.metadata来设置新的元数据

您可以覆盖gymnasium.Wrapper.step() 、 gymnasium.Wrapper.render() 、 gymnasium.Wrapper.close()等。

如果这样做，您可以通过访问属性env来访问传递给包装器的环境（它仍然可能被包装在其他包装器中）。


我们还看一下这个案例的一个例子。大多数 MuJoCo 环境返回由不同terms组成的奖励：例如，可能有一个term奖励代理完成任务，而另一个term则惩罚大型操作（即能源使用）。通常，您可以在环境初始化期间传递这些项的权重参数。然而， Reacher不允许你这样做！尽管如此，奖励的所有单独条款都会在info中返回，因此让我们为 Reacher 构建一个包装器，以允许我们对这些条款进行加权：


In [7]:
class ReacherRewardWrapper(Wrapper):
    def __init__(self, env, reward_dist_weight, reward_ctrl_weight):
        super().__init__(env)
        self.reward_dist_weight = reward_dist_weight
        self.reward_ctrl_weight = reward_ctrl_weight

    def step(self, action):
        obs, _, terminated, truncated, info = self.env.step(action)
        reward = (
            self.reward_dist_weight * info["reward_dist"]
            + self.reward_ctrl_weight * info["reward_ctrl"]
        )
        return obs, reward, terminated, truncated, info